In [1]:
import os
from dotenv import load_dotenv
import time
import gradio as gr
from openai import OpenAI
load_dotenv(override=True)

gemini_api_key = os.getenv("GOOGLE_API_KEY")
multi_model_api_key=os.getenv("Multiple_Model_Key")

In [5]:


# =========================
# Model IDs
# =========================
MISRAL_MODEL = "mistralai/devstral-2512:free"
NVIDIA_MODEL = "nvidia/nemotron-3-nano-30b-a3b:free"
GEMINI_MODEL = "gemini-2.5-flash-lite"

SYSTEM_MESSAGE = "You are a snarky assistant"

# =========================
# LLM Calls
# =========================
def call_misral(prompt):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=multi_model_api_key
    )
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=MISRAL_MODEL,
        messages=messages
    )
    return response.choices[0].message.content


def call_nvidia(prompt):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=multi_model_api_key
    )
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=NVIDIA_MODEL,
        messages=messages
    )
    return response.choices[0].message.content


def call_gemini(prompt):
    client = OpenAI(
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        api_key=gemini_api_key
    )
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=GEMINI_MODEL,
        messages=messages
    )
    return response.choices[0].message.content

In [6]:
def stream_text(text, chunk_size=80):
    for i in range(0, len(text), chunk_size):
        yield text[:i + chunk_size]
        time.sleep(0.02)

def format_response(model, text):
    return f"""
### 🤖 {model} Response

{text}
"""

def stream_model(prompt, model):
    if model == "Gemini":
        raw_text = call_gemini(prompt)
    elif model == "Nvidia":
        raw_text = call_nvidia(prompt)
    elif model == "Misral":
        raw_text = call_misral(prompt)
    else:
        raise ValueError("Unknown model")

    formatted = format_response(model, raw_text)

    for chunk in stream_text(formatted):
        yield chunk

In [7]:
css = """
.markdown {
    font-size: 16px;
    line-height: 1.6;
    padding: 10px;
}
.markdown h3 {
    color: #ff7a18;
}
"""
    

# =========================
# Gradio UI
# =========================
message_input = gr.Textbox(
    label="Your message",
    info="Enter a message for the LLM",
    lines=7
)

model_selector = gr.Dropdown(
    ["Gemini", "Nvidia", "Misral"],
    label="Select model",
    value="Gemini"
)

message_output = gr.Markdown(label="Response")

app = gr.Interface(
    fn=stream_model,
    title="LLMs Playground",
    inputs=[message_input, model_selector],
    outputs=message_output,
    css=css,
    examples=[
        ["Explain the Transformer architecture to a layperson", "Gemini"],
        ["Explain the Transformer architecture to an aspiring AI engineer", "Nvidia"]
    ],
    flagging_mode="never"
)


app.launch(auth=("Prat", "abcde"))


C:\Users\Pratyush.Gupta\AppData\Local\anaconda3\Lib\site-packages\gradio\interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  super().__init__(


* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
